In [1]:
import pandas as pd
import requests
from datetime import datetime

In [19]:
def query(target, server, start, end, aggType):
    # targets_req = []

    # for target in targets:
    #     data = {}
    #     data["payload"] = {}
    #     data["payload"]["schema"] = server
    #     # data["payload"]["additional"] = [aggType]
    #     data["target"] = target
    #     targets_req.append(data)

    url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query"
    data = {
        "range": {
            "from": start,
            "to": end,
        },
        "targets": [
        {
          "payload": {
            "schema": server,
            "additional": [
                    aggType,
                ]
          },
          "target": target
        }
      ],
    }
    response = requests.post(url, json=data)
    print(response)
    print(response.json())
    print(len(response.json()))

    master = pd.json_normalize(response.json(), record_path="datapoints", meta=["target"]).rename(columns={0: "value", 1: "timestamp"}).set_index("target").rename_axis(None)
    # Remove the rows where the metric is None (i.e., do not show the averaged rows because this is not useful)
    # master = master[~master["metric"].isna()]
    master["timestamp"] = master["timestamp"].astype("datetime64[ms]").map(lambda x: x.to_pydatetime().replace(tzinfo=tz.tzutc()).astimezone(tz.tzlocal()))

    return master

In [20]:
target=["#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_441_control/hood_sash"]

In [21]:
query(target=target,
                server="biotech_main",
                start=str(datetime(2024, 1, 1)),
                end=str(datetime.now()),
                aggType="aggD"
                )

<Response [400]>
{'error': 'Error requesting REST API'}
1


KeyError: "Key 'datapoints' not found. If specifying a record_path, all elements of data should have the path."